<a href="https://colab.research.google.com/github/nekoniii3/Colabo_Samples/blob/main/026_Moore_AnimateAnyone/Moore_AnimateAnyone_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **※最初にランタイムからタイプをGPUに変更して下さい。**

# **①Moore_AnimateAnyoneインストールなど**

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/Moore-AnimateAnyone
%cd /content/Moore-AnimateAnyone

!pip install -q gradio==3.50.2 diffusers==0.24.0 av==11.0.0 decord==0.6.0 einops==0.4.1 accelerate==0.21.0
!pip install -q omegaconf==2.2.3
!pip install -q https://github.com/openai/CLIP/archive/d50d76daa670286dd6cacf3bcd80b5e4823fc8e1.zip

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateAnyone/raw/main/stable-diffusion-v1-5/model_index.json -d /content/Moore-AnimateAnyone/pretrained_weights/stable-diffusion-v1-5 -o model_index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateAnyone/raw/main/stable-diffusion-v1-5/unet/config.json -d /content/Moore-AnimateAnyone/pretrained_weights/stable-diffusion-v1-5/unet -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateAnyone/resolve/main/stable-diffusion-v1-5/unet/diffusion_pytorch_model.bin -d /content/Moore-AnimateAnyone/pretrained_weights/stable-diffusion-v1-5/unet -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateAnyone/raw/main/stable-diffusion-v1-5/v1-inference.yaml -d /content/Moore-AnimateAnyone/pretrained_weights/stable-diffusion-v1-5 -o v1-inference.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateAnyone/raw/main/stable-diffusion-v1-5/feature_extractor/preprocessor_config.json -d /content/Moore-AnimateAnyone/pretrained_weights/stable-diffusion-v1-5/feature_extractor -o preprocessor_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/patrolli/AnimateAnyone/resolve/main/denoising_unet.pth -d /content/Moore-AnimateAnyone/pretrained_weights -o denoising_unet.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/patrolli/AnimateAnyone/resolve/main/motion_module.pth -d /content/Moore-AnimateAnyone/pretrained_weights -o motion_module.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/patrolli/AnimateAnyone/resolve/main/pose_guider.pth -d /content/Moore-AnimateAnyone/pretrained_weights -o pose_guider.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/patrolli/AnimateAnyone/resolve/main/reference_unet.pth -d /content/Moore-AnimateAnyone/pretrained_weights -o reference_unet.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse/raw/main/config.json -d /content/Moore-AnimateAnyone/pretrained_weights/sd-vae-ft-mse -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.bin -d /content/Moore-AnimateAnyone/pretrained_weights/sd-vae-ft-mse -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.safetensors -d /content/Moore-AnimateAnyone/pretrained_weights/sd-vae-ft-mse -o diffusion_pytorch_model.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lambdalabs/sd-image-variations-diffusers/raw/main/image_encoder/config.json -d /content/Moore-AnimateAnyone/pretrained_weights/image_encoder -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lambdalabs/sd-image-variations-diffusers/resolve/main/image_encoder/pytorch_model.bin -d /content/Moore-AnimateAnyone/pretrained_weights/image_encoder -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateAnyone/resolve/main/dw-ll_ucoco_384.onnx -d /content/Moore-AnimateAnyone/pretrained_weights/DWPose -o dw-ll_ucoco_384.onnx
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateAnyone/resolve/main/yolox_l.onnx -d /content/Moore-AnimateAnyone/pretrained_weights/DWPose -o yolox_l.onnx


# **②Moore_AnimateAnyone起動**

In [ ]:
!python app.py

# **（補足）Motion Sequence作成アプリ起動<br>※①の後に行ってください（②は停止させる）**

## **1.追加インストール**

In [ ]:
!pip install controlnet_aux
!pip install onnxruntime-gpu

## **2.モーション作成アプリ起動**

In [ ]:
import os
import gradio as gr
from src.dwpose import DWposeDetector
from pathlib import Path
from src.utils.util import get_fps, read_frames, save_videos_from_pil
import numpy as np

def process_video(input_video_path):

    print(input_video_path)
    video_name = os.path.splitext(os.path.basename(input_video_path))[0]

    dir_path = "vid2pose"
    os.makedirs(dir_path, exist_ok=True)
    # path = os.getcwd()
    out_path = os.path.join(dir_path, video_name + "_kps.mp4")
    #configs/inference/pose_videos/man.mp4

    detector = DWposeDetector()
    detector = detector.to(f"cuda")

    fps = get_fps(input_video_path)
    frames = read_frames(input_video_path)
    kps_results = []
    for i, frame_pil in enumerate(frames):
        result, score = detector(frame_pil)
        score = np.mean(score, axis=-1)

        kps_results.append(result)

    print(out_path)
    save_videos_from_pil(kps_results, out_path, fps=fps)

    return out_path

iface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(label="Input Video"),
    outputs=gr.Video(label="Output Video"),
    title="Video"
)
iface.queue()
iface.launch(share=True, inline=False, debug=True)